# Factory Planning I

## Objective and Prerequisites

Want to learn how to create an optimal production plan that will maximize your profits? In this example, we’ll teach you how to solve this classic production planning problem.

More information on this type of model can be found in example # 3 of the fifth edition of Modeling Building in Mathematical Programming by H. P. Williams on pages 255 – 256 and 300 – 302.

This modeling example is at the intermediate level, where we assume that you know Python and are familiar with the Gurobi Python API. In addition, you should have some knowledge about building mathematical optimization models.

**Download the Repository** <br />
You can download the repository containing this and other examples by clicking [here](https://github.com/Gurobi/modeling-examples/archive/master.zip).

---
## Problem Description

A factory makes seven products (Prod 1 to Prod 7) using a range of machines including:

- Four grinders
- Two vertical drills
- Three horizontal drills
- One borer
- One planer

Each product has a defined profit contribution per unit sold (defined as the sales price per unit minus the cost of raw materials). In addition, the manufacturing of each product requires a certain amount of time on each machine (in hours). The contribution and manufacturing time value are shown below. A dash indicates that the manufacturing process for the given product does not require that machine.

| <i></i> | PROD1 | PROD2 | PROD3 | PROD4 | PROD5 | PROD6 | PROD7 |
| --- | --- | --- | --- | --- | --- | --- | --- |
| Profit | 10 | 6 | 8 | 4 | 11 | 9 | 3 |
| Grinding | 0.5 | 0.7 | - | - | 0.3 | 0.2 | 0.5 |
| Vertical Drilling | 0.1 | 0.2 | - | 0.3 | - | 0.6 | - |
| Horizontal Drilling | 0.2 | - | 0.8 | - | - | - | 0.6 |
| Boring | 0.05 | 0.03 | - | 0.07 | 0.1 | - | 0.08 |
| Planning | - | - | 0.01 | - | 0.05 | - | 0.05 |

In each of the six months covered by this model, one or more of the machines is scheduled to be down for maintenance and as a result will not be available to use for production that month. The maintenance schedule is as follows:

| Month | Machine |
| --- | --- |
| January | One grinder |
| February | Two horizontal drills |
| March | One borer |
| April | One vertical drill |
| May | One grinder and one vertical drill |
| June | One horizontal drill |

There are limitations on how many of each product can be sold in a given month. These limits are shown below:

| Month | PROD1 | PROD2 | PROD3 | PROD4 | PROD5 | PROD6 | PROD7 |
| --- | --- | --- | --- | --- | --- | --- | --- |
| January | 500 | 1000 | 300 | 300 | 800 | 200 | 100 |
| February | 600 | 500 | 200 | 0 | 400 | 300 | 150 |
| March | 300 | 600 | 0 | 0 | 500 | 400 | 100 |
| April | 200 | 300 | 400 | 500 | 200 | 0 | 100 |
| May | 0 | 100 | 500 | 100 | 1000 | 300 | 0 |
| June | 500 | 500 | 100 | 300 | 1100 | 500 | 60 |

Up to 100 units of each product may be stored in inventory at a cost of $\$0.50$ per unit per month. At the start of January, there is no product inventory. However, by the end of June, there should be 50 units of each product in inventory.

The factory produces products six days a week using two eight-hour shifts per day. It may be assumed that each month consists of 24 working days. Also, for the purposes of this model, there are no production sequencing issues that need to be taken into account.

What should the production plan look like? Also, is it possible to recommend any price increases and determine the value of acquiring any new machines?

This problem is based on a larger model built for the Cornish engineering company of Holman Brothers.

---
## Model Formulation

### Sets and Indices

$t \in \text{Months}=\{\text{Jan},\text{Feb},\text{Mar},\text{Apr},\text{May},\text{Jun}\}$: Set of months.

$p \in \text{Products}=\{1,2,\dots,7\}$: Set of products.

$m \in \text{Machines}=\{\text{Grinder},\text{VertDrill},\text{horiDrill},\text{Borer},\text{Planer}\}$: Set of machines.

### Parameters

$\text{hours_per_month} \in \mathbb{R}^+$: Time (in hours/month) available at any machine on a monthly basis. It results from multiplying the number of working days (24 days) by the number of shifts per day (2) by the duration of a shift (8 hours).

$\text{max_inventory} \in \mathbb{N}$: Maximum number of units of a single product type that can be stored in inventory at any given month.

$\text{holding_cost} \in \mathbb{R}^+$: Monthly cost (in USD/unit/month) of keeping in inventory a unit of any product type.

$\text{store_target} \in \mathbb{N}$: Number of units of each product type to keep in inventory at the end of the planning horizon.

$\text{profit}_p \in \mathbb{R}^+$: Profit (in USD/unit) of product $p$.

$\text{installed}_m \in \mathbb{N}$: Number of machines of type $m$ installed in the factory.

$\text{down}_{t,m} \in \mathbb{N}$: Number of machines of type $m$ scheduled for maintenance at month $t$.

$\text{time_req}_{m,p} \in \mathbb{R}^+$: Time (in hours/unit) needed on machine $m$ to manufacture one unit of product $p$.

$\text{max_sales}_{t,p} \in \mathbb{N}$: Maximum number of units of product $p$ that can be sold at month $t$.


### Decision Variables

$\text{make}_{t,p} \in \mathbb{R}^+$: Number of units of product $p$ to manufacture at month $t$.

$\text{store}_{t,p} \in [0, \text{max_inventory}] \subset \mathbb{R}^+$: Number of units of product $p$ to store at month $t$.

$\text{sell}_{t,p} \in [0, \text{max_sales}_{t,p}] \subset \mathbb{R}^+$: Number of units of product $p$ to sell at month $t$.

**Assumption:** We can produce fractional units.

### Objective Function

- **Profit:** Maximize the total profit (in USD) of the planning horizon.

\begin{equation}
\text{Maximize} \quad Z = \sum_{t \in \text{Months}}\sum_{p \in \text{Products}}
(\text{profit}_p*\text{make}_{t,p} - \text{holding_cost}*\text{store}_{t,p})
\tag{0}
\end{equation}

### Constraints

- **Initial Balance:** For each product $p$, the number of units produced should be equal to the number of units sold plus the number stored (in units of product).

\begin{equation}
\text{make}_{\text{Jan},p} = \text{sell}_{\text{Jan},p} + \text{store}_{\text{Jan},p} \quad \forall p \in \text{Products}
\tag{1}
\end{equation}

- **Balance:** For each product $p$, the number of units produced in month $t$ and the ones previously stored should be equal to the number of units sold and stored in that month (in units of product).

\begin{equation}
\text{store}_{t-1,p} + \text{make}_{t,p} = \text{sell}_{t,p} + \text{store}_{t,p} \quad \forall (t,p) \in \text{Months} \setminus \{\text{Jan}\} \times \text{Products}
\tag{2}
\end{equation}

- **Inventory Target:** The number of units of product $p$ kept in inventory at the end of the planning horizon should hit the target (in units of product).

\begin{equation}
\text{store}_{\text{Jun},p} = \text{store_target} \quad \forall p \in \text{Products}
\tag{3}
\end{equation}
- **Machine Capacity:** Total time used to manufacture any product at machine type $m$ cannot exceed its monthly capacity (in hours).

\begin{equation}
\sum_{p \in \text{Products}}\text{time_req}_{m,p}*\text{make}_{t,p} \leq \text{hours_per_month}*(\text{installed}_m - \text{down}_{t,m}) \quad \forall (t,m) \in \text{Months} \times \text{Machines}
\tag{4}
\end{equation}

---
## Python Implementation

We import the Gurobi Python Module and other Python libraries.

In [27]:
%pip install gurobipy

In [28]:
import numpy as np
import pandas as pd

import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.11 & Gurobi 11.0

### Input Data
We define all the input data of the model.

In [29]:
# Parameters

products = ["Prod1", "Prod2", "Prod3", "Prod4", "Prod5", "Prod6", "Prod7"]
machines = ["grinder", "vertDrill", "horiDrill", "borer", "planer"]
months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun"]

profit = {"Prod1":10, "Prod2":6, "Prod3":8, "Prod4":4, "Prod5":11, "Prod6":9, "Prod7":3}

time_req = {
    "grinder": {    "Prod1": 0.5, "Prod2": 0.7, "Prod5": 0.3,
                    "Prod6": 0.2, "Prod7": 0.5 },
    "vertDrill": {  "Prod1": 0.1, "Prod2": 0.2, "Prod4": 0.3,
                    "Prod6": 0.6 },
    "horiDrill": {  "Prod1": 0.2, "Prod3": 0.8, "Prod7": 0.6 },
    "borer": {      "Prod1": 0.05,"Prod2": 0.03,"Prod4": 0.07,
                    "Prod5": 0.1, "Prod7": 0.08 },
    "planer": {     "Prod3": 0.01,"Prod5": 0.05,"Prod7": 0.05 }
}


# number of machines down
down = {("Jan","grinder"): 1, ("Feb", "horiDrill"): 2, ("Mar", "borer"): 1,
        ("Apr", "vertDrill"): 1, ("May", "grinder"): 1, ("May", "vertDrill"): 1,
        ("Jun", "planer"): 1, ("Jun", "horiDrill"): 1}

# number of each machine available
installed = {"grinder":4, "vertDrill":2, "horiDrill":3, "borer":1, "planer":1}

# market limitation of sells
max_sales = {
    ("Jan", "Prod1") : 500,
    ("Jan", "Prod2") : 1000,
    ("Jan", "Prod3") : 300,
    ("Jan", "Prod4") : 300,
    ("Jan", "Prod5") : 800,
    ("Jan", "Prod6") : 200,
    ("Jan", "Prod7") : 100,
    ("Feb", "Prod1") : 600,
    ("Feb", "Prod2") : 500,
    ("Feb", "Prod3") : 200,
    ("Feb", "Prod4") : 0,
    ("Feb", "Prod5") : 400,
    ("Feb", "Prod6") : 300,
    ("Feb", "Prod7") : 150,
    ("Mar", "Prod1") : 300,
    ("Mar", "Prod2") : 600,
    ("Mar", "Prod3") : 0,
    ("Mar", "Prod4") : 0,
    ("Mar", "Prod5") : 500,
    ("Mar", "Prod6") : 400,
    ("Mar", "Prod7") : 100,
    ("Apr", "Prod1") : 200,
    ("Apr", "Prod2") : 300,
    ("Apr", "Prod3") : 400,
    ("Apr", "Prod4") : 500,
    ("Apr", "Prod5") : 200,
    ("Apr", "Prod6") : 0,
    ("Apr", "Prod7") : 100,
    ("May", "Prod1") : 0,
    ("May", "Prod2") : 100,
    ("May", "Prod3") : 500,
    ("May", "Prod4") : 100,
    ("May", "Prod5") : 1000,
    ("May", "Prod6") : 300,
    ("May", "Prod7") : 0,
    ("Jun", "Prod1") : 500,
    ("Jun", "Prod2") : 500,
    ("Jun", "Prod3") : 100,
    ("Jun", "Prod4") : 300,
    ("Jun", "Prod5") : 1100,
    ("Jun", "Prod6") : 500,
    ("Jun", "Prod7") : 60,
}

holding_cost = 0.5
max_inventory = 100
store_target = 50
hours_per_month = 2*8*24

## Model Deployment
We create a model and the variables. For each product (seven kinds of products) and each time period (month), we will create variables for the amount of which products get manufactured, held, and sold. In each month, there is an upper limit on the amount of each product that can be sold. This is due to market limitations.

In [30]:
factory = gp.Model('Factory Planning I')

make = factory.addVars(months, products, name="Make") # quantity manufactured
store = factory.addVars(months, products, ub=max_inventory, name="Store") # quantity stored
sell = factory.addVars(months, products, ub=max_sales, name="Sell") # quantity sold

Next, we insert the constraints. The balance constraints ensure that the amount of product that is in  storage in the prior month plus the amount that gets manufactured equals the amount that is sold and held for each product in the current month. This ensures that all products in the model are manufactured in some month. The initial storage is empty.

In [31]:
#1. Initial Balance
Balance0 = factory.addConstrs((make[months[0], product] == sell[months[0], product]
                  + store[months[0], product] for product in products), name="Initial_Balance")

#2. Balance
Balance = factory.addConstrs((store[months[months.index(month) -1], product] +
                make[month, product] == sell[month, product] + store[month, product]
                for product in products for month in months
                if month != months[0]), name="Balance")

The Inventory Target constraints force that at the end of the last month the storage contains the specified amount of each product.

In [32]:
#3. Inventory Target
TargetInv = factory.addConstrs((store[months[-1], product] == store_target for product in products),  name="End_Balance")

The capacity constraints ensure that, for each month, the time all products require on a certain kind of machine is less than or equal to the available hours for that type of machine in that month multiplied by the number of available machines in that period. Each product requires some machine hours on different machines. Each machine is down in one or more months due to maintenance, so the number and type of available machines varies per month. There can be multiple machines per machine type.

In [33]:
#4. Machine Capacity

MachineCap = factory.addConstrs((gp.quicksum(time_req[machine][product] * make[month, product]
                             for product in time_req[machine])
                    <= hours_per_month * (installed[machine] - down.get((month, machine), 0))
                    for machine in machines for month in months),
                   name = "Capacity")

The objective is to maximize the profit of the company, which consists of
the profit for each product minus the cost for storing the unsold products. This can be stated as:

In [34]:
#0. Objective Function
obj = gp.quicksum(profit[product] * sell[month, product] -  holding_cost * store[month, product]
               for month in months for product in products)

factory.setObjective(obj, GRB.MAXIMIZE)

Next, we start the optimization and Gurobi finds the optimal solution.

In [35]:
factory.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 79 rows, 126 columns and 288 nonzeros
Model fingerprint: 0x2625f534
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [5e-01, 1e+01]
  Bounds range     [6e+01, 1e+03]
  RHS range        [5e+01, 2e+03]
Presolve removed 74 rows and 110 columns
Presolve time: 0.01s
Presolved: 5 rows, 16 columns, 21 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.2466500e+05   3.640000e+02   0.000000e+00      0s
Extra simplex iterations after uncrush: 1
       3    9.3715179e+04   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds (0.00 work units)
Optimal objective  9.371517857e+04


---
## Analysis

The result of the optimization model shows that the maximum profit we can achieve is $\$93,715.18$.
Let's see the solution that achieves that optimal result.

### Production Plan

This plan determines the amount of each product to make at each period of the planning horizon. For example, in February we make 700 units of product Prod1.

In [36]:
rows = months.copy()
columns = products.copy()
make_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for month, product in make.keys():
    if (abs(make[month, product].x) > 1e-6):
        make_plan.loc[month, product] = np.round(make[month, product].x, 1)
make_plan

,Prod1,Prod2,Prod3,Prod4,Prod5,Prod6,Prod7
Jan,500.0,888.6,382.5,300.0,800.0,200.0,0.0
Feb,700.0,600.0,117.5,0.0,500.0,300.0,250.0
Mar,0.0,0.0,0.0,0.0,0.0,400.0,0.0
Apr,200.0,300.0,400.0,500.0,200.0,0.0,100.0
May,0.0,100.0,600.0,100.0,1100.0,300.0,100.0
Jun,550.0,550.0,0.0,350.0,0.0,550.0,0.0


### Sales Plan

This plan defines the amount of each product to sell at each period of the planning horizon. For example, in February we sell 600 units of product Prod1.

In [37]:
rows = months.copy()
columns = products.copy()
sell_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for month, product in sell.keys():
    if (abs(sell[month, product].x) > 1e-6):
        sell_plan.loc[month, product] = np.round(sell[month, product].x, 1)
sell_plan

,Prod1,Prod2,Prod3,Prod4,Prod5,Prod6,Prod7
Jan,500.0,888.6,300.0,300.0,800.0,200.0,0.0
Feb,600.0,500.0,200.0,0.0,400.0,300.0,150.0
Mar,100.0,100.0,0.0,0.0,100.0,400.0,100.0
Apr,200.0,300.0,400.0,500.0,200.0,0.0,100.0
May,0.0,100.0,500.0,100.0,1000.0,300.0,0.0
Jun,500.0,500.0,50.0,300.0,50.0,500.0,50.0


### Inventory Plan

This plan reflects the amount of product in inventory at the end of each period of the planning horizon. For example, at the end of February we have 100 units of Prod1 in inventory.

In [38]:
rows = months.copy()
columns = products.copy()
store_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for month, product in store.keys():
    if (abs(store[month, product].x) > 1e-6):
        store_plan.loc[month, product] = np.round(store[month, product].x, 1)
store_plan

,Prod1,Prod2,Prod3,Prod4,Prod5,Prod6,Prod7
Jan,0.0,0.0,82.5,0.0,0.0,0.0,0.0
Feb,100.0,100.0,0.0,0.0,100.0,0.0,100.0
Mar,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Apr,0.0,0.0,0.0,0.0,0.0,0.0,0.0
May,0.0,0.0,100.0,0.0,100.0,0.0,100.0
Jun,50.0,50.0,50.0,50.0,50.0,50.0,50.0


**Note:** If you want to write your solution to a file, rather than print it to the terminal, you can use the model.write() command. An example implementation is:

`factory.write("factory-planning-1-output.sol")`

---
## References

H. Paul Williams, Model Building in Mathematical Programming, fifth edition.

Copyright &copy; 2020 Gurobi Optimization, LLC

Slack Variable Calculation


In [39]:
def calculate_slack(make, store, sell, time_requirements, machine_availability, installed_machines,
                    max_sales, max_inventory, store_target, months, products):
    slack_values = {}

    # 1. Slack for Machine Time Constraint
    for m in months:
        lhs = sum(time_requirements[p] * make.get((m, p), 0) for p in products)
        rhs = machine_availability[m] * installed_machines
        slack_values[f"machine_capacity_{m}"] = rhs - lhs

    # 2. Slack for Market Demand Constraint
    for m in months:
        for p in products:
            lhs = sell.get((m, p), 0)
            rhs = max_sales[m, p]
            slack_values[f"market_demand_{m}_{p}"] = rhs - lhs

    # 3. Slack for Inventory Capacity Constraint
    for m in months:
        for p in products:
            lhs = store.get((m, p), 0)
            rhs = max_inventory
            slack_values[f"storage_capacity_{m}_{p}"] = rhs - lhs

    # 4. Slack for End-of-Period Inventory Target
    for p in products:
        lhs = store.get(('Jun', p), 0)
        rhs = store_target
        slack_values[f"final_inventory_{p}"] = rhs - lhs

    return slack_values

# Example Usage:
# Replace these dictionaries with actual values from your optimal solution
optimal_make = {('Jan', 'P1'): 500, ('Jan', 'P2'): 400, ('Feb', 'P1'): 600, ('Feb', 'P2'): 450}
optimal_store = {('Jan', 'P1'): 100, ('Jan', 'P2'): 50, ('Feb', 'P1'): 120, ('Feb', 'P2'): 60}
optimal_sell = {('Jan', 'P1'): 500, ('Jan', 'P2'): 400, ('Feb', 'P1'): 600, ('Feb', 'P2'): 450}

slack_results = calculate_slack(optimal_make, optimal_store, optimal_sell,
                                time_requirements={'P1': 2, 'P2': 3},
                                machine_availability={'Jan': 0.95, 'Feb': 0.90},
                                installed_machines=1000,
                                max_sales={('Jan', 'P1'): 500, ('Jan', 'P2'): 400,
                                           ('Feb', 'P1'): 600, ('Feb', 'P2'): 450},
                                max_inventory=500,
                                store_target=100,
                                months=['Jan', 'Feb'],
                                products=['P1', 'P2'])

print(slack_results)


{'machine_capacity_Jan': -1250.0, 'machine_capacity_Feb': -1650.0, 'market_demand_Jan_P1': 0, 'market_demand_Jan_P2': 0, 'market_demand_Feb_P1': 0, 'market_demand_Feb_P2': 0, 'storage_capacity_Jan_P1': 400, 'storage_capacity_Jan_P2': 450, 'storage_capacity_Feb_P1': 380, 'storage_capacity_Feb_P2': 440, 'final_inventory_P1': 100, 'final_inventory_P2': 100}


Shadow price calculation

In [40]:
model = Model("Factory_Optimization")

# Define decision variables
months = ["Jan", "Feb"]
products = ["P1", "P2"]

make = model.addVars(months, products, name="make", vtype=GRB.CONTINUOUS)
store = model.addVars(months, products, name="store", vtype=GRB.CONTINUOUS)
sell = model.addVars(months, products, name="sell", vtype=GRB.CONTINUOUS)

# Given parameters
time_req = {"P1": 2, "P2": 3}
machine_avail = {"Jan": 0.95, "Feb": 0.90}
installed_machines = 1000
max_sales = {("Jan", "P1"): 500, ("Jan", "P2"): 400, ("Feb", "P1"): 600, ("Feb", "P2"): 450}
max_inventory = 500
store_target = 100

# Adding Machine Capacity Constraints
for m in months:
    model.addConstr(sum(time_req[p] * make[m, p] for p in products) <= machine_avail[m] * installed_machines,
                    name=f"machine_capacity_{m}")

# Adding Market Demand Constraints
for m in months:
    for p in products:
        model.addConstr(sell[m, p] <= max_sales[m, p], name=f"market_demand_{m}_{p}")

# Adding Storage Capacity Constraints
for m in months:
    for p in products:
        model.addConstr(store[m, p] <= max_inventory, name=f"storage_capacity_{m}_{p}")

# Adding End-of-Period Inventory Constraint
for p in products:
    model.addConstr(store["Feb", p] == store_target, name=f"final_inventory_{p}")

# Objective Function: Maximize Profit
profit_per_unit = {"P1": 50, "P2": 70}  # Assume profit values
holding_cost = 5  # Assume storage cost per unit

model.setObjective(sum(profit_per_unit[p] * sell[m, p] - holding_cost * store[m, p]
                       for m in months for p in products), GRB.MAXIMIZE)

# Solve the model
model.optimize()

# Function to extract shadow prices
def extract_shadow_prices(model):
    shadow_prices = {}

    if model.status == GRB.OPTIMAL:
        for constraint in model.getConstrs():
            shadow_prices[constraint.constrName] = constraint.pi  # Dual value (shadow price)

    return shadow_prices

# Extract and display shadow prices
shadow_prices = extract_shadow_prices(model)
print("\nShadow Prices (Dual Values):")
for constraint, value in shadow_prices.items():
    print(f"{constraint}: {value:.4f}")

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 12 rows, 12 columns and 14 nonzeros
Model fingerprint: 0x15d89db9
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [5e+00, 7e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 1e+03]
Presolve removed 12 rows and 12 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.1350000e+05   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.135000000e+05

Shadow Prices (Dual Values):
machine_capacity_Jan: 0.0000
machine_capacity_Feb: 0.0000
market_demand_Jan_P1: 50.0000
market_demand_Jan_P2: 70.0000
market_demand_Feb_P1: 50.0000
market_demand_Feb

Sensitivity Analysis

In [41]:
# Function to perform sensitivity analysis
def sensitivity_analysis(model):
    print("\nSensitivity Analysis Results:\n")
    for constraint in model.getConstrs():
        shadow_price = constraint.pi  # Shadow Price
        lower_bound = constraint.SARHSLow  # Lowest RHS value before basis change
        upper_bound = constraint.SARHSUp  # Highest RHS value before basis change

        print(f"Constraint: {constraint.constrName}")
        print(f"  Shadow Price: {shadow_price:.4f}")
        print(f"  RHS Sensitivity Range: ({lower_bound}, {upper_bound})")
        print("-" * 50)

# Run sensitivity analysis
sensitivity_analysis(model)


Sensitivity Analysis Results:

Constraint: machine_capacity_Jan
  Shadow Price: 0.0000
  RHS Sensitivity Range: (0.0, inf)
--------------------------------------------------
Constraint: machine_capacity_Feb
  Shadow Price: 0.0000
  RHS Sensitivity Range: (0.0, inf)
--------------------------------------------------
Constraint: market_demand_Jan_P1
  Shadow Price: 50.0000
  RHS Sensitivity Range: (0.0, inf)
--------------------------------------------------
Constraint: market_demand_Jan_P2
  Shadow Price: 70.0000
  RHS Sensitivity Range: (0.0, inf)
--------------------------------------------------
Constraint: market_demand_Feb_P1
  Shadow Price: 50.0000
  RHS Sensitivity Range: (0.0, inf)
--------------------------------------------------
Constraint: market_demand_Feb_P2
  Shadow Price: 70.0000
  RHS Sensitivity Range: (0.0, inf)
--------------------------------------------------
Constraint: storage_capacity_Jan_P1
  Shadow Price: 0.0000
  RHS Sensitivity Range: (0.0, inf)
-----------